<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Schema for Semi-Structured Data
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Example and Exercise on Schema<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Read JSON file using default schema</li>
        <li>Read JSON file using user defined schema</li>
        <li>Exercise 1: Use inferred schema to read log files</li>
        <li>Exercise 2: Create user define schema for a particular column and read log files</li>
        <li>Exercise 3: Create user define schema for all fields in a particular column and read log files</li>
        <li>Compare performance between using default schema and user defined schema</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Schema Handling").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Schemas with Semi-Structured Data e.g. JSON Data<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
    <ul>
        <li><b>Tabular Data: </b> such as data from .csv files or relational databases have a structured format where each column in the dataset has a valid datatype eventhough some values might be NULL</li>
        <li><b>Semi-Structured Data: </b> does not conform to a formal data model, a given column might have zero, one or many values.</li>
        <li>JSON data has key value pairs defined is an example of such Semi-Structured Data</li>
    </ul>
</span>

In [ ]:
if DATA_BRICKS == False:
    jsonfile = "../datasets/zips.json"
else:
    jsonfile = "dbfs:/FileStore/datasets/zips.json"

In [ ]:
zipDF = spark.read.json(jsonfile)

In [ ]:
showDataFrame(zipDF)

### Interpreting Schema

In [ ]:
zipDF.printSchema()

<span style="font-family:verdana; font-size:1.0em;">
    Store the schema as an object by calling <b>`.schema`</b> on a DataFrame. Schemas consist of a <i>`StructType`</i>, which is a collection of <b><i>`StructField`s</i></b>.  Each <i>`StructField`</i> gives a name and a type for a given field in the data.
</span>

In [ ]:
zipschema = zipDF.schema
[print(field) for field in zipschema]

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
User Defined Schemas<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Spark will interpret the schema as defined above. However there are challenges with such inferred schemas:
    <ul>
        <li>Inference means Spark has to scan the whole dataset (creating extra work), thus impacting performance</li>
        <li>Defining our own schema helps with changing datatypes e.g. change Long to Integer</li>
        <li>Leaving out fields in data that might not be of interest to processing</li>
    </ul>
</span>

### Use StructFields which contains StructType<br>
<span style="font-family:verdana; font-size:1.0em;">
Interpret the population as integer type (4 byte value), in the inferred schema it was long (8 byte value). Population of any country will fit in 4 byte value (2,417,483,647)
The location data in the zip codes dataset can be treated as FloatType
</span>

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, FloatType

zipsSchema = StructType([
  StructField("city", StringType(), True), 
  StructField("loc", 
    ArrayType(FloatType(), True), True),
  StructField("pop", IntegerType(), True)
])

In [ ]:
zipDF1 = (spark.read
    .schema(zipsSchema)
    .json(jsonfile))

In [ ]:
showDataFrame(zipDF1)

## Compare performance with and without schema

In [ ]:
%timeit aDF = spark.read.schema(zipsSchema).json(jsonfile).count()

In [ ]:
%timeit aDF = spark.read.json(jsonfile).count()

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 1: Inferred Schema<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Data is from <a href="https://archive.ics.uci.edu/ml/datasets/UbiqLog+(smartphone+lifelogging)">Smartphone data from UCI Repository</a> It is open source project data that can be used to analytics and machine learning.<br>
Perform the following steps:
    <ol>
        <li>Import the data from the UCILog directory (all log*.txt files) into Spark DataFrame</li>
        <li>Print the schema of the DataFrame</li>
        <li>Observe the printed Schema - cateogories, nested datastructures, etc</li>
    </ol>
</span>

In [ ]:
if DATA_BRICKS == False:
    jsonfile = "../datasets/UCILog/log*"
else:
    jsonfile = "dbfs:/FileStore/datasets/UCILog/log*"

<span style="font-family:verdana; font-size:1.0em;">
Schema shows:
    <ol>
        <li>Six different categories - Application, Bluetooth ....</li>
        <li>SMS has nested datastructure</li>
        <li>Checkout the field _corrupt_record</li>
    </ol>
</span>

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 2: Create User Defined Schema<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Define schema that will only import the <b>SMS</b>.
Perform the following steps:<br>
    <ol>
        <li>Import appropritate classes required (defined below (Package & Function))</li>
        <li>Create schema to import only SMS field - import it as StringType</li>
        <li>Only import records from SMS field that are <b>not NULL</b></li>
    </ol>
</span>

| Package | Function |
|---------|---------|
| `pyspark.sql.types` | `StructType`, `StructField`, `StringType` |
| `pyspark.sql.functions` | `col` |

In [ ]:
# import appropriate classes
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

<font color='tomato'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Exercise 3: Create Schema for All Fields in SMS<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
Define schema that will only import the <b>SMS</b>.
Perform the following steps:<br>
    <ol>
        <li>Create schema to import only SMS field - define for all fields in SMS</li>
        <li>include fields Address, data, metadata and metadata.name</li>
        <li>Only import records from SMS field that are <b>not NULL</b></li>
        <li>Display the dataframe</li>
    </ol>
</span>